# Credit Risk Resampling Techniques

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import ClusterCentroids
from sklearn.preprocessing import StandardScaler

In [10]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


# Split the Data into Training and Testing

In [11]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X = pd.get_dummies(X)

# Create our target
y = df['loan_status'].to_frame()

In [12]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,homeowner_mortgage,homeowner_own,homeowner_rent
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.497472,0.398911,0.103616
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.499997,0.489678,0.304764
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000,0.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000,0.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,1.000000,1.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000,1.000000,1.000000


In [14]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [15]:
# Create X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [16]:
# Create the StandardScaler instance

scaler = StandardScaler()

In [17]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [18]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Simple Logistic Regression

In [19]:

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [31]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [32]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]])

In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [36]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)

# View the count of target classes with Counter
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)
Counter(y_resampled['loan_status'])

Counter({'low_risk': 56277, 'high_risk': 56277})

In [45]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', max_iter=100, random_state=1) 
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [46]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.9946414201183431

In [47]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[  622,     3],
       [  111, 18648]])

In [48]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [49]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled['loan_status'])

Counter({'low_risk': 56277, 'high_risk': 56277})

In [52]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs',max_iter=100, random_state=1) 
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [53]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [54]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]])

In [55]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [74]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled['loan_status'])

Counter({'high_risk': 1875, 'low_risk': 1875})

In [75]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [76]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9932813049736127

In [77]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  620,     5],
       [  102, 18657]])

In [78]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [68]:
# Resample the training data with SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled['loan_status'])

Counter({'high_risk': 55652, 'low_risk': 55885})

In [69]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [71]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9946680739911509

In [72]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  622,     3],
       [  110, 18649]])

In [73]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   Random Oversampling and Combination (Over and Under) Sampling using SMOTEENN Algorithm had the best Balanced Accuracy Score of 0.9946680739911509.

2. Which model had the best recall score?

    All the examined models (LogisticRegression without resampling, Random Oversampling, SMOTE Oversampling, Cluster Centroids Undersampling, and Combination (Over and Under) Sampling using SMOTEENN Algorithm) had the same Average 99% Recall Score.

3. Which model had the best geometric mean score?

    The examined models (Random Oversampling, SMOTE Oversampling, Cluster Centroids Undersampling, and Combination (Over and Under) Sampling using SMOTEENN Algorithm) had the same Average 99% Geometric Score.

